## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Global stability of the simple pendulum via SOS

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pydrake.all import MathematicalProgram, Solve, Variables
from pydrake.symbolic import Polynomial
matplotlib.rcParams['figure.figsize'] = (10, 10)

In [ ]:
prog = MathematicalProgram()

# Declare the indeterminates.
s = prog.NewIndeterminates(1, 's')[0]
c = prog.NewIndeterminates(1, 'c')[0]
td = prog.NewIndeterminates(1, 'td')[0]
x = np.array([s, c, td])

# Write out the dynamics in terms of sin(theta), cos(theta), and thetadot.
f = [c * td, - s * td, - td - s]

# The fixed-point in this coordinate (because cos(0)=1).
x0 = np.array([0, 1, 0])

# Construct a polynomial v that contains all monomials with s,c,thetadot up
# to degree 2.
deg_v = 4
v = prog.NewFreePolynomial(Variables(x), deg_v).ToExpression()

# Add a constraint to enforce that v is strictly positive away from x0.
# (Note that because our coordinate system is sine and cosine, V is also zero
# at theta=2pi, etc).
eps = 1e-5
prog.AddSosConstraint(v - eps * (x - x0).dot(x - x0))

# Construct the polynomial which is the time derivative of V.
vdot = v.Jacobian(x).dot(f)

# Construct a polynomial L representing the "Lagrange multiplier".
deg_l = 2
l = prog.NewFreePolynomial(Variables(x), deg_l).ToExpression()

# Add a constraint that Vdot is strictly negative away from x0 (but make an
# exception for the upright fixed point by multipling by s^2).
prog.AddSosConstraint(- vdot - l * (s**2 + c**2 - 1) - eps *
                      (x - x0).dot(x - x0) * s**2)

# Enforce V(0) = 0.
prog.AddLinearConstraint(v.Substitute({s: 0, c: 1, td: 0}) == 0)

# Add v(theta=pi) = 1, just to set the scale.
prog.AddLinearConstraint(v.Substitute({s: 1, c: 0, td: 0}) == 1)

# Solve the SOS program.
result = Solve(prog)
assert result.is_success()
v_sol = Polynomial(result.GetSolution(v))

# Compare Lyapunov function with mechanical energy

In [ ]:
# Evaluate energy.
qlim = (-2 * np.pi, 2 * np.pi)
qdlim = (-2, 2)
q = np.linspace(*qlim, 201)
qd = np.linspace(*qdlim, 201)
Q, QD = np.meshgrid(q, qd)
energy = .5 * QD**2 + (1 - np.cos(Q))

# Evaluate Lyapunov.
vplot = Q.copy()
env = {}
for i in range(len(q)):
    for j in range(len(qd)):
        env[s] = np.sin(Q[i, j])
        env[c] = np.cos(Q[i, j])
        env[td] = QD[i, j]
        vplot[i, j] = v_sol.Evaluate(env)
        
# Plot Lyapunov and energy.
plt.contour(Q, QD, vplot)
plt.contour(Q, QD, energy, alpha=0.5, linestyles='--')

# Misc plot settings.
plt.xlabel(r'$\theta$')
plt.ylabel(r'$\dot \theta$')
plt.title(r'Lyapunov function (solid), Mechanical Energy (dashed)')

In [ ]:
# Evaluate vector field.
def f2d(x):
    return [x[1], - x[1] - np.sin(x[0])]
Q, QD = np.meshgrid(q, qd)
Qd, QDd = f2d([Q, QD])

# Color the streamlines according to the magnitude of f(x).
color = np.sqrt(Qd**2 + QDd**2)

# Vector field.
strm = plt.streamplot(Q[0], QD.T[0], Qd, QDd, color=color.T)

# Plot Lyapunov.
plt.contour(Q, QD, vplot, colors='r')

# Misc plot settings.
plt.gca().set_aspect('equal')
plt.xlabel(r'$\theta$')
plt.ylabel(r'$\dot \theta$')
plt.title(r'Lyapunov function (red), vector field (colors)')